## Kernel Interface Proposal 

When a user uses a javascript kernel, the last evaluation, is inspected
  
```js
> x
<representation>
```

`x` can be inspected at first with `util.inspect` as well as some smarts to convert certain kinds of objects to rich data for an `execute_result`.

If a user wants to send something more rich for display, they can import the `display` function

```js
import { display, updateDisplay } from 'inodejs';
```

Then the interface should be as js as possible

```
display(obj, options)
```

where options can include `display_id`, `update`, etc. We'd want a way to set metadata or transient as well

In [1]:
var util = require('util')

In [2]:
function raw_display(o) {
    // Pretend this is sending it on the iopub socket at display_data
    console.log(o);
}

function display(obj, opts={}) {
    // opts will accept these keys:
    
    // raw: boolean // obj is expected to be a mimebundle dict
    // metadata: object, raw
    // transient: object, raw
    // display_id: string - if specified, folded into transient
    
    const data = opts.raw ? obj : { 'text/plain': util.inspect(obj) };
    const content = Object.assign({},
      {
        data,
        metadata: opts.metadata || {},
        transient: opts.transient || {}
      }
    );
    
    if(opts.display_id) {
        content.transient = Object.assign({}, content.transient, { display_id: opts.display_id })
    }
    
    return content
}

In [6]:
display(2)

{ data: { 'text/plain': '2' }, metadata: {}, transient: {} }

In [3]:
display(2, {display_id: 235})

{ data: { 'text/plain': '2' },
  metadata: {},
  transient: { display_id: 235 } }

In [4]:
display(2, {transient: {model_id: 124}, display_id: 235})

{ data: { 'text/plain': '2' },
  metadata: {},
  transient: { model_id: 124, display_id: 235 } }

In [5]:
display(2, {transient: {model_id: 124}})

{ data: { 'text/plain': '2' },
  metadata: {},
  transient: { model_id: 124 } }